In [1]:
import openml
import numpy as np

dataset = openml.datasets.get_dataset(487)
X, y, _, attribute_names = dataset.get_data(target=dataset.default_target_attribute)
from sklearn.preprocessing import StandardScaler

sc = StandardScaler()
X = sc.fit_transform(X)
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

C:\Users\elsag\OneDrive\Desktop\Uni\Year 4\Academic\Dissertation\Final Report\Code\scikit-learn-1.2.1\scikit-learn-1.2.1\venv\lib\site-packages\openml\datasets\functions.py:438: FutureWarning: Starting from Version 0.15 `download_data`, `download_qualities`, and `download_features_meta_data` will all be ``False`` instead of ``True`` by default to enable lazy loading. To disable this message until version 0.15 explicitly set `download_data`, `download_qualities`, and `download_features_meta_data` to a bool while calling `get_dataset`.
  warnings.warn(


In [14]:
from tensorflow import keras

In [15]:
def fit_bnn_model(model, loss, num_epochs, learning_rate, X_train, y_train):

    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=learning_rate, beta_1=0.9, beta_2=0.999, epsilon=1e-8, clipnorm=1),
        loss=loss,
        metrics=[keras.metrics.MeanSquaredError()],
    )

    model.fit(x=X_train, y=y_train, epochs=num_epochs)

In [29]:
from uncertainty_analysis import bnn_helper_functions as bnn

hidden_units = [10, 5]
learning_rate = 0.5
num_epochs = 500
input_shape = X_train.shape[1]
train_size = len(X_train)

predictions = []
for i in range(0, 10):
    model = bnn.bnn_model(hidden_units, input_shape, train_size, activation='tanh')
    fit_bnn_model(model, bnn.negative_loglikelihood, num_epochs, learning_rate, X_train, y_train)
    predictions.append(model.predict(X_test))

Epoch 1/500
1/1 [==============================] - 3s 3s/step - loss: 720.3954 - mean_squared_error: 223.5662
Epoch 2/500
1/1 [==============================] - 0s 0s/step - loss: 293.5435 - mean_squared_error: 219.3517
Epoch 3/500
1/1 [==============================] - 0s 16ms/step - loss: 150.5993 - mean_squared_error: 211.6534
Epoch 4/500
1/1 [==============================] - 0s 0s/step - loss: 106.6356 - mean_squared_error: 182.8099
Epoch 5/500
1/1 [==============================] - 0s 0s/step - loss: 116.3181 - mean_squared_error: 175.2907
Epoch 6/500
1/1 [==============================] - 0s 16ms/step - loss: 88.4503 - mean_squared_error: 152.3667
Epoch 7/500
1/1 [==============================] - 0s 0s/step - loss: 35.6818 - mean_squared_error: 156.4725
Epoch 8/500
1/1 [==============================] - 0s 16ms/step - loss: 143.5688 - mean_squared_error: 153.4646
Epoch 9/500
1/1 [==============================] - 0s 0s/step - loss: 53.3518 - mean_squared_error: 151.6699
Epoch 1

In [31]:
from uncertainty_analysis import project_helper_functions as phf

best_model_results = predictions[phf.bf_get_best_model_idx(predictions, y_test)]
phf.write_bnn_model_results(best_model_results, "paper_bnn_results.txt")